In [1]:
import torch
import jieba
import pandas as pd

data_dir = 'https://mirror.coggle.club/dataset/coggle-competition/'
train_data = pd.read_csv(data_dir + 'intent-classify/train.csv', sep='\t', header=None)
test_data = pd.read_csv(data_dir + 'intent-classify/test.csv', sep='\t', header=None)

In [2]:
train_data.head()

,0,1
0,还有双鸭山到淮阴的汽车票吗13号的,Travel-Query
1,从这里怎么回家,Travel-Query
2,随便播放一首专辑阁楼里的佛里的歌,Music-Play
3,给看一下墓王之王嘛,FilmTele-Play
4,我想看挑战两把s686打突变团竞的游戏视频,Video-Play


In [3]:
train_data = train_data.sample(frac=1.0)

In [4]:
train_data[1], lbl = pd.factorize(train_data[1])

In [5]:
def coustom_data_iter(texts, labels):
    for x, y in zip(texts, labels):
        yield x, y

In [6]:
train_iter = coustom_data_iter(train_data[0].values[:], train_data[1].values[:])

In [7]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = jieba.lcut


def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.602 seconds.
Prefix dict has been built successfully.


In [8]:
vocab.get_itos()[:10]

['<unk>', '的', '我', '一下', '播放', '是', '吗', '给', '帮', '一个']

In [9]:
vocab(['我', '一下', '今天'])

[2, 3, 41]

In [10]:
# from gensim.models import KeyedVectors

# # 需要自行下载，然后修改路径后运行
# wv_from_text = KeyedVectors.load_word2vec_format('/home/lyz/work/dataset/词向量/tencent-ailab-embedding-zh-d100-v0.2.0-s/tencent-ailab-embedding-zh-d100-v0.2.0-s.txt', binary=False)

# pretrained_w2v = []
# for w in vocab.get_itos():
#     if w in wv_from_text:
#         pretrained_w2v.append(wv_from_text[w])
#     else:
#         pretrained_w2v.append(np.random.rand(100))
        
# pretrained_w2v = np.vstack(pretrained_w2v)

In [11]:
def text_pipeline(x): return vocab(tokenizer(x))

In [12]:
processed_text = torch.tensor(text_pipeline('今天我们在这里'), dtype=torch.int64)

In [13]:
from torch.nn.utils.rnn import pad_sequence

In [14]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

# 90%的文本长度都在20个单词以内
def collate_batch(batch, max_len=20):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        processed_text = F.pad(processed_text, pad=[0, max_len,], mode='constant', value=0)
        if len(processed_text) > max_len:
            processed_text = processed_text[:max_len]

        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list).T
    return label_list.to(device), text_list.to(device)

In [15]:
class BILSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size):
        super(BILSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        # self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_w2v))
        
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True)
        self.hidden2label = torch.nn.Linear(hidden_dim*2, label_size)
        

    def forward(self, sentence):
        # torch.Size([16, 20])
        # print(sentence.shape)
        sentence = torch.transpose(sentence, 1, 0)
        
        # torch.Size([20, 16])
        # print(sentence.shape)
        
        # torch.Size([20, 16, 100])
        x = self.embeddings(sentence)
        # print(x.shape)
        
        # 5 seqence length
        # 3 batch size
        # 10 input size
        # input = torch.randn(5, 3, 10)

        lstm_out, self.hidden = self.lstm(x)
        # torch.Size([20, 16, 128])
        # print(lstm_out.shape)
        y = self.hidden2label(lstm_out[-1,:,:])
        return y

In [16]:
torch.transpose(torch.rand((2,3,1)), 0, 2).shape

torch.Size([1, 3, 2])

In [17]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [18]:
num_class = len(lbl)
vocab_size = len(vocab)
emsize = 100
model = BILSTM(vocab_size, emsize, 64, num_class).to(device)

In [19]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 40  # epoch
LR = 2 # learning rate
BATCH_SIZE = 16  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5.0, gamma=0.75)
total_accu = None

train_iter = coustom_data_iter(train_data[0].values[:], train_data[1].values[:])
train_dataset = to_map_style_dataset(train_iter)

num_train = int(len(train_dataset) * 0.75)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

In [20]:
for (label, text) in train_dataloader:
    break
    
model(text).shape
# 12, 16, 100
# sequence length * batch size * embedding dim

torch.Size([16, 12])

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))

In [ ]:
test_iter = coustom_data_iter(test_data[0].values[:], [0] * len(test_data))
test_dataset = to_map_style_dataset(test_iter)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, collate_fn=collate_batch)

In [ ]:
def predict(dataloader):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text).argmax(1)
            test_pred += list(predicted_label.cpu().numpy())
    return test_pred

In [ ]:
test_pred = predict(test_dataloader)
test_pred = [lbl[x] for x in test_pred]

In [ ]:
pd.DataFrame({
    'ID': range(1, len(test_pred) + 1),
    'Target': test_pred,
}).to_csv('nlp_submit.csv', index=None)

# 提交一下吧~